# Sockets Examples

## Sample to get the host and ipaddress of the machine

In [2]:
import socket


def print_machine_info():
    host_name = socket.gethostname()
    ip_address = socket.gethostbyname(host_name)
    print ("Host name: %s" % host_name)
    print ("IP address: %s" % ip_address)
    
    
if __name__ == '__main__':
    print_machine_info()

Host name: localhost
IP address: 127.0.0.1


## UDP Sending 

In [3]:
import socket

UDP_IP = "127.0.0.1"
UDP_PORT = 5005
MESSAGE = "Hello, World!"

print ("UDP target IP:", UDP_IP)
print ("UDP target port:", UDP_PORT)
print( "message:", MESSAGE)

sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.sendto(MESSAGE.encode(), (UDP_IP, UDP_PORT))

UDP target IP: 127.0.0.1
UDP target port: 5005
message: Hello, World!


13

```python
import socket

UDP_IP = "127.0.0.1"
UDP_PORT = 5005

sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.bind((UDP_IP, UDP_PORT))

while True:  # Wait for ever :), till we receive `bye`
    data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
    print ("received message:", data)
    if data.lower() == "bye":
        break
```

## TCP

I am assuming you already know about TCP, if not, please read about them and come back. 

But if you do'nt have the time then, here is the gist of TCP communication:

- Server program starts wait for any client request at a particular address and port
- Client program starts and connects to the server on the designated address and port to initiate the conversation.
- Client program sends a request.
- Server program responds according to client requests.
- Previous two steps goes in a loop till one of them disconnects.

### TCP Server

Below is a sample TCP Server code.

```python
# server_1.py
import socket
import time


# create a socket object
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# get local machine name
host = socket.gethostname()
port = 9999
# bind to the port
serversocket.bind((host, port))
# queue up to 5 requests
serversocket.listen(5)

while True:
    # establish a connection
    clientsocket, addr = serversocket.accept()
    print("Got a connection from %s" % str(addr))
    currentTime = time.ctime(time.time()) + "\r\n"
    clientsocket.send(currentTime.encode('ascii'))
    clientsocket.close()
```

**Output:**
```
Got a connection from ('127.0.0.1', 42586)
```

### TCP Client

```python
# client_1.py
import socket


# create a socket object
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# get local machine name
host = socket.gethostname()
port = 9999
# connection to hostname on the port.
s.connect((host, port))
# Receive no more than 1024 bytes
tm = s.recv(1024)
s.close()
print("The time got from the server is %s" % tm.decode('ascii'))
```

### NTP Clients

In [5]:
from contextlib import closing
from socket import socket, AF_INET, SOCK_DGRAM
import sys
import struct
import time


def ntp_time(host="pool.ntp.org", port=123):
    NTP_PACKET_FORMAT = "!12I"
    NTP_DELTA = 2208988800 # 1970-01-01 00:00:00
    NTP_QUERY = '\x1b' + 47 * '\0'  

    with closing(socket( AF_INET, SOCK_DGRAM)) as s:
        s.sendto(bytes(NTP_QUERY, 'utf-8'), (host, port))
        msg, address = s.recvfrom(1024)
    unpacked = struct.unpack(NTP_PACKET_FORMAT,
                   msg[0:struct.calcsize(NTP_PACKET_FORMAT)])
    t = unpacked[10] + float(unpacked[11]) / 2**32 - NTP_DELTA
    return time.ctime(t).replace("  "," ")


if __name__ == "__main__":
    print(ntp_time())

Mon Jun 3 08:00:27 2019
